In [8]:
""" use kernel tensorflow_p36 - NOT TF 2"""
!pip install --upgrade tensorflow==1.8.0

     |████████████████████████████████| 49.1 MB 389 kB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 42.0 MB/s eta 0:00:01
     |████████████████████████████████| 889 kB 43.3 MB/s eta 0:00:01
  Created wheel for html5lib: filename=html5lib-0.9999999-py3-none-any.whl size=107220 sha256=6fd43f9f71cac0cac7ec3158d355019ef8eef7473cd24e97e2bcd7adde91ef29
  Stored in directory: /home/ec2-user/.cache/pip/wheels/90/1c/cb/a87fd097ff74648ecc468a703001f6c7c86d8a71d459e65c98
Successfully built html5lib
ERROR: tensorflow-serving-api 1.15.0 has requirement tensorflow~=1.15.0, but you'll have tensorflow 1.8.0 which is incompatible.
ERROR: tensorflow-gpu 1.15.3 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-gpu 1.15.3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.8.0 which is incompatible.
ERROR: tensorflow-gpu 1.15.3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.3.0 whic

In [9]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

EPOCHS = 20
IMG_WIDTH = 224
IMG_HEIGHT = 224
NUM_CATEGORIES = 2
TEST_SIZE = 0.2

In [10]:
def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    images = []
    labels=[]
    counter = 0
    for labelname in os.listdir(data_dir):
        path= data_dir + "/" + labelname 
        for pic in os.listdir(path):
            path2= data_dir + "/" + labelname + "/" + pic
            img= cv2.imread(path2)
            #print ("path- ", path)
            #print ("path2- ", path2)
            re_img=cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
            images.append(re_img)
            labels.append(labelname)
            counter= counter+1
    print(counter) 
    return (images,labels)


def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
        """
# Create a convolutional neural network
    model = tf.keras.models.Sequential([

         # Convolutional layer. Learn 32 filters using a 3x3 kernel
         tf.keras.layers.Conv2D(
             32, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(4, 4)),
         tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv2D(
             64, (3, 3), activation="relu", padding = "same", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
         ),


         # Max-pooling layer, using 3x3 pool size
         tf.keras.layers.MaxPooling2D(pool_size=(3, 3)),
         tf.keras.layers.Dropout(0.2),
         

         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
        
        

         # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*32, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        
         # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*32, activation="relu"),
         tf.keras.layers.Dropout(0.2),
         
         # Flatten units
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dropout(0.2),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        
        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*16, activation="relu"),
         tf.keras.layers.Dropout(0.2),
        

        # Add a hidden layer with dropout
         tf.keras.layers.Dense(NUM_CATEGORIES*8, activation="relu"),
         tf.keras.layers.Dropout(0.2),
         
         # Add an output layer with number of categories
         tf.keras.layers.Dense(NUM_CATEGORIES, activation="softmax")
     ])

# Train neural network
    model.compile(
         optimizer="adam",
         loss="categorical_crossentropy",
         metrics=["accuracy"]
   )

    return model



In [11]:
# Get image arrays and labels for all image files
    images, labels = load_data("/home/ec2-user/SageMaker/BrainScans/brain")

    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE)


253


In [12]:
model = get_model()


In [13]:
import time

start = time.time()

# Fit model on training data
model_history = model.fit(x_train, y_train, epochs=EPOCHS)

end = time.time()
print(end - start)

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential_1/conv2d_2/Relu (defined at <ipython-input-13-4c83784ca44c>:6) ]] [Op:__inference_train_function_3501]

Function call stack:
train_function


In [6]:
   # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

51/51 - 0s - loss: 0.4696 - acc: 0.7843


[0.46960273209740133, 0.78431374]

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 56, 56, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

In [8]:
model.save("model-latest")